In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv(r'C:/Users/ניב/Downloads/fifa20.csv')
#df = pd.read_csv(r'C:/Users/oshri/Desktop/FifaProject/fifa20.csv')
df.head(7)

,Name,Club,Nation,Age,Height,Weight,Foot,Wage,Value,Position,...,Composure,Heading Acc.,Interceptions,Marking,Stand Tackle,Slide Tackle,Aggression,Jumping,Stamina,Strength
0,Lionel Messi,FC Barcelona,Argentina,31,"5' 7""",72kg,Left,£500K,£95M,"RW, ST, CF",...,96,70,40,33,37,26,48,68,75,68
1,Cristiano Ronaldo,Piemonte Calcio,Portugal,33,"6' 2""",83kg,Right,£320K,£61.5M,"ST, LW",...,95,89,29,28,32,24,63,95,85,78
2,Neymar,Paris Saint-Germain,Brazil,26,"5' 9""",68kg,Right,£230K,£95M,"LW, CAM",...,94,62,36,35,30,29,51,61,81,49
3,Virgil van Dijk,Liverpool,Netherlands,27,"6' 4""",92kg,Right,£175K,£74M,CB,...,89,87,90,93,93,86,83,90,75,92
4,Jan Oblak,Atlético de Madrid,Slovenia,25,"6' 2""",87kg,Right,£83K,£66.5M,GK,...,68,15,19,27,12,18,34,78,41,78
5,Kevin De Bruyne,Manchester City,Belgium,27,"5' 11""",70kg,Right,£250K,£82M,"CAM, CM",...,91,55,63,68,62,53,76,63,89,74
6,Robert Lewandowski,Bayern München,Poland,30,"6' 0""",80kg,Right,£210K,£75.5M,ST,...,86,85,39,34,42,19,81,84,78,85


In [3]:
df.columns

Index(['Name', 'Club', 'Nation', 'Age', 'Height', 'Weight', 'Foot', 'Wage',
       'Value', 'Position', 'Rating', 'Potential', 'Acceleration',
       'Sprint Speed', 'Positioning', 'Finishing', 'Long Shots', 'Penalties',
       'Shot Power', 'Volleys', 'Crossing', 'Curve', 'FK. Accuracy',
       'Long Pass', 'Short Pass.', 'Vision', 'Agility', 'Balance',
       'Ball Control', 'Dribbling', 'Reactions', 'Composure', 'Heading Acc.',
       'Interceptions', 'Marking', 'Stand Tackle', 'Slide Tackle',
       'Aggression', 'Jumping', 'Stamina', 'Strength'],
      dtype='object')

In [4]:
df.isnull().sum().head(10)

Name           0
Club        2517
Nation        22
Age            0
Height         0
Weight         0
Foot           0
Wage           0
Value          0
Position       0
dtype: int64

In [5]:
df['Club'].fillna('Free player',inplace=True)   
df['Nation'].fillna('Other',inplace=True)  
df.isnull().sum().head(10)

Name        0
Club        0
Nation      0
Age         0
Height      0
Weight      0
Foot        0
Wage        0
Value       0
Position    0
dtype: int64

In [6]:
df.dtypes.head(10)

Name        object
Club        object
Nation      object
Age          int64
Height      object
Weight      object
Foot        object
Wage        object
Value       object
Position    object
dtype: object

In [7]:
df['Weight']=df['Weight'].apply(lambda x:x.replace('kg',''))
df["Weight"] = pd.to_numeric(df["Weight"])
df.dtypes.head(10)

Name        object
Club        object
Nation      object
Age          int64
Height      object
Weight       int64
Foot        object
Wage        object
Value       object
Position    object
dtype: object

In [8]:
df['Position']

0         RW, ST, CF
1             ST, LW
2            LW, CAM
3                 CB
4                 GK
            ...     
17890             ST
17891    CF, CAM, ST
17892        CM, CAM
17893         CM, RB
17894             CB
Name: Position, Length: 17895, dtype: object

In [9]:
df['Position'] = df['Position'].str.rsplit(',').str[0] 
df['Position']

0        RW
1        ST
2        LW
3        CB
4        GK
         ..
17890    ST
17891    CF
17892    CM
17893    CM
17894    CB
Name: Position, Length: 17895, dtype: object

In [10]:
df['Height']

0         5' 7"
1         6' 2"
2         5' 9"
3         6' 4"
4         6' 2"
          ...  
17890     5' 9"
17891     5' 7"
17892     5' 9"
17893     5' 8"
17894    5' 10"
Name: Height, Length: 17895, dtype: object

In [11]:
df['Height']=df['Height'].apply(lambda x:x.replace('"',''))
df['Height']=df['Height'].apply(lambda x:x.replace("'",''))
df['Height']=df['Height'].apply(lambda x:x.replace(' ',''))
df["Height"] = pd.to_numeric(df["Height"])
df['Height']=((df['Height']//10)*12+df['Height']%10)*2.54

df.loc[df.Height==1554.48,'Height']=177.80
df.loc[df.Height==1557.02,'Height']=180.34


df['Height'] = df['Height'].apply(np.int64)

df['Height'].head(5)

0    170
1    187
2    175
3    193
4    187
Name: Height, dtype: int64

In [12]:
df['Value']

0          £95M
1        £61.5M
2          £95M
3          £74M
4        £66.5M
          ...  
17890     £220K
17891     £210K
17892     £275K
17893     £240K
17894     £275K
Name: Value, Length: 17895, dtype: object

In [13]:
df['Wage']

0        £500K
1        £320K
2        £230K
3        £175K
4         £83K
         ...  
17890    £1.5K
17891     £650
17892    £1.3K
17893     £601
17894    £1.1K
Name: Wage, Length: 17895, dtype: object

In [14]:
def value_to_int(fifa_value):
    try:
        value = int(float(fifa_value[1:-1]))
        money = fifa_value[-1:]

        if money == 'M':
            value = value * 1000000
        elif money == 'K':
            value = value * 1000
    except ValueError:
        value = 0
    return value

df['Value'] =df['Value'].apply(value_to_int)
df['Wage'] = df['Wage'].apply(value_to_int)

In [15]:
df.head()

,Name,Club,Nation,Age,Height,Weight,Foot,Wage,Value,Position,...,Composure,Heading Acc.,Interceptions,Marking,Stand Tackle,Slide Tackle,Aggression,Jumping,Stamina,Strength
0,Lionel Messi,FC Barcelona,Argentina,31,170,72,Left,500000,95000000,RW,...,96,70,40,33,37,26,48,68,75,68
1,Cristiano Ronaldo,Piemonte Calcio,Portugal,33,187,83,Right,320000,61000000,ST,...,95,89,29,28,32,24,63,95,85,78
2,Neymar,Paris Saint-Germain,Brazil,26,175,68,Right,230000,95000000,LW,...,94,62,36,35,30,29,51,61,81,49
3,Virgil van Dijk,Liverpool,Netherlands,27,193,92,Right,175000,74000000,CB,...,89,87,90,93,93,86,83,90,75,92
4,Jan Oblak,Atlético de Madrid,Slovenia,25,187,87,Right,83000,66000000,GK,...,68,15,19,27,12,18,34,78,41,78


In [16]:
df.describe()

,Age,Height,Weight,Wage,Value,Rating,Potential,Acceleration,Sprint Speed,Positioning,...,Composure,Heading Acc.,Interceptions,Marking,Stand Tackle,Slide Tackle,Aggression,Jumping,Stamina,Strength
count,17895.000000,17895.000000,17895.000000,17895.000000,1.789500e+04,17895.000000,17895.000000,17895.000000,17895.000000,17895.000000,...,17895.000000,17895.000000,17895.000000,17895.000000,17895.000000,17895.000000,17895.000000,17895.000000,17895.000000,17895.000000
mean,24.843141,180.631014,75.442246,7092.444258,2.261004e+06,67.864096,71.817938,65.359151,65.528751,52.155798,...,60.024811,53.932272,48.242693,48.627941,49.178430,46.921319,57.660017,65.745236,64.675272,66.604191
std,4.491318,6.753862,7.040194,16626.706564,4.985435e+06,5.419857,5.890638,14.631078,14.351086,19.098419,...,10.958922,16.937762,20.541587,19.831196,21.420497,21.115792,16.865001,11.774002,15.162360,11.950686
min,15.000000,154.000000,50.000000,0.000000,0.000000e+00,59.000000,59.000000,12.000000,11.000000,2.000000,...,12.000000,5.000000,3.000000,3.000000,5.000000,4.000000,9.000000,17.000000,13.000000,16.000000
25%,21.000000,175.000000,70.000000,1000.000000,4.000000e+05,64.000000,67.000000,58.000000,58.000000,42.000000,...,54.000000,47.000000,27.000000,31.000000,29.000000,26.000000,47.000000,59.000000,59.000000,60.000000
50%,25.000000,180.000000,75.000000,2000.000000,7.250000e+05,67.000000,71.000000,68.000000,68.000000,58.000000,...,61.000000,58.000000,56.000000,55.000000,58.000000,55.000000,61.000000,67.000000,68.000000,68.000000
75%,28.000000,185.000000,80.000000,6000.000000,2.000000e+06,71.000000,76.000000,76.000000,75.000000,65.500000,...,67.000000,65.000000,65.000000,65.000000,67.000000,64.000000,70.000000,74.000000,74.500000,75.000000
max,41.000000,205.000000,110.000000,500000.000000,9.500000e+07,94.000000,95.000000,97.000000,96.000000,95.000000,...,96.000000,93.000000,92.000000,94.000000,93.000000,90.000000,95.000000,95.000000,97.000000,97.000000


In [17]:
df.shape

(17895, 41)

In [18]:
df.to_csv(r'C:/Users/ניב/Downloads/cleanData.csv',index=False)
#df.to_csv(r'C:/Users/oshri/Desktop/FifaProject/cleanData.csv',index=False)